In [1]:
# Install these packages if running from colab
!pip install tensorflow-datasets --quiet
!pip install pydot --quiet
!pip install transformers --quiet

# install huggingface datasets
!pip install datasets --quiet

! pip install rouge-score nltk --quiet
! pip install huggingface_hub --quiet

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re

#let's make longer output readable without scrolling
from pprint import pprint

# the toxic parallel dataset, with rouge metric
from datasets import load_dataset, load_from_disk, load_metric, DatasetDict

In [3]:
metric = load_metric("rouge")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [4]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
dataset_path = 'drive/MyDrive/Colab Notebooks/w266_project_data'

# define output CSV paths
csv_path = 'drive/MyDrive/Colab Notebooks/w266_project_predictions/'

In [6]:
# a list of model's output CSV file
output_file_names = ['bart_large_zsl.csv', 'bart_xsum_zsl.csv', 'bart_cnn_zsl.csv', 
                     'bart_large_ft.csv', 'bart_xsum_ft.csv', 'bart_cnn_ft.csv',
                     't5_large_zsl.csv', 't5_large_ft.csv']

In [7]:
# load the dataset
dataset = load_from_disk(dataset_path)

In [8]:
for output_file_name in output_file_names:
  df_bart_predictions = pd.read_csv(csv_path + output_file_name)
  rouge_results = metric.compute(predictions=df_bart_predictions['test_predictions'],
                                 references=df_bart_predictions['test_references'])
  display(output_file_name)
  display(pd.concat({k: pd.DataFrame(v) for k, v in rouge_results.items()}))
  print()

'bart_large_zsl.csv'

precision    recall  fmeasure
rouge1    0   0.688862  0.781581  0.725401
          1   0.701717  0.795253  0.738163
          2   0.715363  0.808488  0.751325
rouge2    0   0.536699  0.613321  0.566110
          1   0.553529  0.631362  0.583172
          2   0.569986  0.649211  0.600422
rougeL    0   0.684956  0.777753  0.721363
          1   0.697833  0.790545  0.734120
          2   0.710884  0.803705  0.746336
rougeLsum 0   0.684903  0.777921  0.721727
          1   0.698026  0.790502  0.734329
          2   0.711092  0.803790  0.746439

'bart_xsum_zsl.csv'

precision    recall  fmeasure
rouge1    0   0.190387  0.272476  0.211892
          1   0.206568  0.289687  0.227558
          2   0.220814  0.305864  0.241901
rouge2    0   0.090913  0.110966  0.094838
          1   0.102912  0.125067  0.106814
          2   0.116779  0.140006  0.120994
rougeL    0   0.180439  0.255980  0.199875
          1   0.195257  0.271972  0.214472
          2   0.210869  0.288119  0.229814
rougeLsum 0   0.180670  0.255481  0.199715
          1   0.195679  0.272802  0.215234
          2   0.209920  0.289156  0.229229

'bart_cnn_zsl.csv'

...

precision    recall  fmeasure
rouge1    0   0.794336  0.815024  0.798662
          1   0.808535  0.827740  0.811992
          2   0.823195  0.841926  0.826543
rouge2    0   0.677240  0.692831  0.680106
          1   0.697077  0.711697  0.699373
          2   0.716843  0.731084  0.718724
rougeL    0   0.790881  0.810776  0.794678
          1   0.805123  0.823776  0.808372
          2   0.818994  0.836338  0.821925
rougeLsum 0   0.791533  0.811051  0.795340
          1   0.804985  0.824025  0.808462
          2   0.819281  0.837718  0.822141

't5_large_zsl.csv'

precision    recall  fmeasure
rouge1    0   0.638141  0.705613  0.656988
          1   0.653749  0.720597  0.671132
          2   0.668410  0.736502  0.684887
rouge2    0   0.487113  0.541158  0.500393
          1   0.503556  0.558238  0.516411
          2   0.521807  0.577966  0.534618
rougeL    0   0.628700  0.695874  0.647017
          1   0.644940  0.711520  0.662371
          2   0.660033  0.725890  0.676511
rougeLsum 0   0.630037  0.695248  0.647414
          1   0.644808  0.710817  0.662359
          2   0.659081  0.726658  0.675565

't5_large_ft.csv'

precision    recall  fmeasure
rouge1    0   0.791314  0.807227  0.793616
          1   0.805579  0.820415  0.806739
          2   0.820787  0.834078  0.820985
rouge2    0   0.671102  0.682769  0.671346
          1   0.691117  0.701535  0.690595
          2   0.711044  0.720925  0.709947
rougeL    0   0.787731  0.802990  0.789254
          1   0.801935  0.816427  0.802899
          2   0.815654  0.829242  0.816012
rougeLsum 0   0.787984  0.803924  0.790094
          1   0.801969  0.816557  0.802882
          2   0.816744  0.829443  0.815850

### examine a few predictions in each model

In [9]:
print('The first 3 example\'s input and label')
pd.DataFrame(dataset['test'][:5])

The first 3 example's input and label


,en_toxic_comment,en_neutral_comment
0,"holy shit , the 61 61 view must be hilarious .",The 6161 view must be funny
1,all she has to do is smile and keep her cool w...,All she has to do is smile and keep her cool w...
2,can we apply that litmus test to muslims that ...,can we apply that litmus test to muslims that ...
3,"thats fucking it , im joining",That's it! I'm joining.
4,the residents of davis have a collective stick...,The residents of Davis are not so pleasant.


In [10]:
# examine a few predictions in each model
for output_file_name in output_file_names:
  df_bart_predictions = pd.read_csv(csv_path + output_file_name)
  display(output_file_name)
  display(df_bart_predictions.head(5))
  print()

'bart_large_zsl.csv'

,test_predictions,test_references
0,"['holy shit , the 61 61 view must be hilarious...",['The 6161 view must be funny']
1,['all she has to do is smile and keep her cool...,['All she has to do is smile and keep her cool...
2,['can we apply that litmus test to muslims tha...,['can we apply that litmus test to muslims tha...
3,"['thats fucking it , im joining']","[""That's it! I'm joining.""]"
4,['the residents of davis have a collective sti...,['The residents of Davis are not so pleasant.']


'bart_xsum_zsl.csv'

,test_predictions,test_references
0,['The BBC has released a video of the moment t...,['The 6161 view must be funny']
1,['Theresa May has been praised for the way she...,['All she has to do is smile and keep her cool...
2,['A few years ago I wrote a post saying that i...,['can we apply that litmus test to muslims tha...
3,"[""Thats what I thought about it, so I'm going ...","[""That's it! I'm joining.""]"
4,"[""The BBC has been given a special report on t...",['The residents of Davis are not so pleasant.']


'bart_cnn_zsl.csv'

...

,test_predictions,test_references
0,['The 61 61 view must be hilarious.'],['The 6161 view must be funny']
1,['all she has to do is smile and keep her cool...,['All she has to do is smile and keep her cool...
2,['can we apply that litmus test to muslims tha...,['can we apply that litmus test to muslims tha...
3,"['thats it , im joining']","[""That's it! I'm joining.""]"
4,['the residents of davis have a collective pro...,['The residents of Davis are not so pleasant.']


't5_large_zsl.csv'

,test_predictions,test_references
0,['the 61 61 view must be hilarious .'],['The 6161 view must be funny']
1,['all she has to do is smile and keep her cool...,['All she has to do is smile and keep her cool...
2,['can we apply that litmus test to muslims tha...,['can we apply that litmus test to muslims tha...
3,['im joining youtube . im joining youtube . im...,"[""That's it! I'm joining.""]"
4,['residents of davis have a collective stick u...,['The residents of Davis are not so pleasant.']


't5_large_ft.csv'

,test_predictions,test_references
0,['The 61 61 view must be hilarious.'],['The 6161 view must be funny']
1,['All she has to do is smile and keep her cool...,['All she has to do is smile and keep her cool...
2,['can we apply that litmus test to muslims tha...,['can we apply that litmus test to muslims tha...
3,"['thats it , im joining']","[""That's it! I'm joining.""]"
4,['The residents of davis have a collective pro...,['The residents of Davis are not so pleasant.']
